This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [1]:
base_directory = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-*/ses-*/roi/'
file_pattern = 'sub-*0mm_sphere_roi_composite.nii.gz'

In [2]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
dv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=file_pattern).run()
dv_df

Attempting to import from: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-*/ses-*/roi//sub-*0mm_sphere_roi_composite.nii.gz


,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Hansen_2018/ses-ERC_BL/roi/sub-Hansen_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Inman_2018/ses-AMY_L/roi/sub-Inman_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Mankin_2021/ses-ERC_R/roi/sub-Mankin_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Halgren_1985/ses-HPC_&_AMY_BL/roi/sub-Halgren_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Turnbul_1985/ses-BNM_BL/roi/sub-Turnbul_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Hampson_2018/ses-HPC_BL_L_R/roi/sub-Hampson_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Oh_2012/ses-ANT_BL/roi/sub-Oh_2012_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Velasco_1987/ses-CM_BL/roi/sub-Velasco_1987_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Velasco_1987/ses-Fornix_L/roi/sub-Velasco_1987_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Lin_2020/ses-Fornix_BL/roi/sub-Lin_2020_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,...,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Coleshill_1990/ses-HPC_L/roi/sub-Coleshill_1990_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Coleshill_1990/ses-HPC_R/roi/sub-Coleshill_1990_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Asahi_2014/ses-STN_BL/roi/sub-Asahi_2014_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Laxton_2010/ses-Fornix_BL/roi/sub-Laxton_2010_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Wang_2022/ses-STN_BL/roi/sub-Wang_2022_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Williams_2011/ses-STN_BL/roi/sub-Williams_2011_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Salanova_2021/ses-ANT_BL/roi/sub-Salanova_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Zhang_2021/ses-BNM_BL/roi/sub-Zhang_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Dujardin_2001/ses-STN_BL/roi/sub-Dujardin_2001_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partner

**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

This Should Often Be Left Default

In [3]:
preceding_id = '/roi/sub-'
proceeding_id = '/sub'

In [4]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
# dv_df = GiiNiiFileImport.splice_colnames(dv_df, preceding_id, proceeding_id)
dv_df

,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Hansen_2018/ses-ERC_BL/roi/sub-Hansen_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Inman_2018/ses-AMY_L/roi/sub-Inman_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Mankin_2021/ses-ERC_R/roi/sub-Mankin_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Halgren_1985/ses-HPC_&_AMY_BL/roi/sub-Halgren_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Turnbul_1985/ses-BNM_BL/roi/sub-Turnbul_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Hampson_2018/ses-HPC_BL_L_R/roi/sub-Hampson_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Oh_2012/ses-ANT_BL/roi/sub-Oh_2012_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Velasco_1987/ses-CM_BL/roi/sub-Velasco_1987_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Velasco_1987/ses-Fornix_L/roi/sub-Velasco_1987_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Lin_2020/ses-Fornix_BL/roi/sub-Lin_2020_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,...,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Coleshill_1990/ses-HPC_L/roi/sub-Coleshill_1990_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Coleshill_1990/ses-HPC_R/roi/sub-Coleshill_1990_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Asahi_2014/ses-STN_BL/roi/sub-Asahi_2014_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Laxton_2010/ses-Fornix_BL/roi/sub-Laxton_2010_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Wang_2022/ses-STN_BL/roi/sub-Wang_2022_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Williams_2011/ses-STN_BL/roi/sub-Williams_2011_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Salanova_2021/ses-ANT_BL/roi/sub-Salanova_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Zhang_2021/ses-BNM_BL/roi/sub-Zhang_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Dujardin_2001/ses-STN_BL/roi/sub-Dujardin_2001_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,/Users/cu135/Partner

# Load Map to Check Damage Within

Import Region of Interest Masks

In [23]:
base_directory = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_06b/conjunction_map'
shared_glob_pattern = 'agreement_Precom_T.nii.gz'

In [24]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
iv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=shared_glob_pattern).run()
iv_df

Attempting to import from: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_06b/conjunction_map/agreement_Precom_T.nii.gz


,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_06b/conjunction_map/agreement_Precom_T.nii.gz
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
902624,0.0
902625,0.0
902626,0.0
902627,0.0


Extract Damage Scores Per Region of Interest

In [25]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [26]:
from calvin_utils.nifti_utils.damage_score_utils import DamageScorer
damage_scorer = DamageScorer(mask_path, dv_df, iv_df)
dmg_df = damage_scorer.calculate_damage_scores()
dmg_df = damage_scorer.sort_dataframes_by_index(dmg_df)
dmg_df

,agreement_Precom_T_spatial_corr,agreement_Precom_T_cosine,agreement_Precom_T_sum,agreement_Precom_T_average_subject_in_target,agreement_Precom_T_average_target_in_subject,agreement_Precom_T_num_in_roi
path,,,,,,
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Hansen_2018/ses-ERC_BL/roi/sub-Hansen_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,0.004502,0.004513,78.923992,0.000562,39.461996,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Inman_2018/ses-AMY_L/roi/sub-Inman_2018_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,-0.000065,-0.000054,-0.940063,-0.000007,-0.470031,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Mankin_2021/ses-ERC_R/roi/sub-Mankin_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,0.001469,0.001477,18.260965,0.000130,18.260965,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Halgren_1985/ses-HPC_&_AMY_BL/roi/sub-Halgren_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,0.007135,0.007154,216.683315,0.001542,36.113886,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Turnbul_1985/ses-BNM_BL/roi/sub-Turnbul_1985_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,-0.000143,-0.000131,-2.299274,-0.000016,-1.149637,0.0
...,...,...,...,...,...,...
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Williams_2011/ses-STN_BL/roi/sub-Williams_2011_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,-0.000435,-0.000424,-7.421073,-0.000053,-3.710537,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Salanova_2021/ses-ANT_BL/roi/sub-Salanova_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,-0.001641,-0.001630,-28.500731,-0.000203,-14.250366,0.0
/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/derivatives/sub-Zhang_2021/ses-BNM_BL/roi/sub-Zhang_2021_MNI152_T1_1.0mm_sphere_roi_composite.nii.gz,-0.000143,-0.000131,-2.299274,-0.000016,-1.149637,0.0


Save the Results

In [27]:
# out_path = '/Volumes/Expansion/datasets/Manitoba_Epilepsy_PET/metadata/nifti_metadata.csv'

In [28]:
# dmg_df.to_csv(out_path)

Save Results into Master CSV

In [29]:
master_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/DBS_studies_influencing_memory/metadata/master_list.csv'
sheet = None

In [30]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
cal_palm = CalvinStatsmodelsPalm(input_csv_path=master_path, output_dir=None, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,study,Memory_Specific,ses,N,Memory_Outcome,Pre_Post_Memory_Effect_Size__Cohen_s_D_,Location,List_of_Coordinates,Side,Frequency__Hz_,...,flipped_lesions_all_datasets_weighted_avg_r_map_pcc_inverse_r_map_sum,flipped_lesions_all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_subject_in_target,flipped_lesions_all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_target_in_subject,flipped_lesions_all_datasets_weighted_avg_r_map_pcc_inverse_r_map_num_in_roi,agreement_pcc_inverse_r_map_spatial_corr,agreement_pcc_inverse_r_map_cosine,agreement_pcc_inverse_r_map_sum,agreement_pcc_inverse_r_map_average_subject_in_target,agreement_pcc_inverse_r_map_average_target_in_subject,agreement_pcc_inverse_r_map_num_in_roi
0,Acera 2019,0.0,STN_BL,50.0,Impaired,-0.180960,STN,"[14.2, -15.1, -6.1], [-13.5, -14.2, -6.1]",BL,NaN,...,0.164717,1.077068e-06,0.082359,0.0,-0.000593,-0.000570,-0.039039,-2.826996e-07,-0.019519,0.0
1,Asahi 2014,0.0,STN_BL,11.0,Improved,0.234145,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,130.0,...,-0.080103,-5.237877e-07,-0.040052,0.0,0.001943,0.001965,0.134648,9.750566e-07,0.067324,0.0
2,Barcia 2022,1.0,Fornix_BL,1.0,Mixed,NaN,Fornix,"[-2, 3.5, -1], [2, 3.5, -1]",BL,NaN,...,-0.802307,-5.246204e-06,-0.401154,0.0,0.004742,0.004764,0.326355,2.363317e-06,0.163178,0.0
3,Bergfeld 2017,0.0,vALIC_BL,20.0,Impaired,-0.688208,vALIC,"[-11.6, 7.2, -3.2], [12.7, 8.7, -2.4]",BL,140.0,...,0.079793,5.217602e-07,0.039897,0.0,-0.000570,-0.000548,-0.037524,-2.717330e-07,-0.018762,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.080103,-5.237877e-07,-0.040052,0.0,0.001943,0.001965,0.134648,9.750566e-07,0.067324,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.164717,1.077068e-06,0.082359,0.0,-0.000593,-0.000570,-0.039039,-2.826996e-07,-0.019519,0.0
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.080103,-5.237877e-07,-0.040052,0.0,0.001943,0.001965,0.134648,9.750566e-07,0.067324,0.0
102,You 2020,0.0,STN_BL,NaN,NaN,NaN,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,NaN,...,-0.080103,-5.237877e-07,-0.040052,0.0,0.001943,0.001965,0.134648,9.750566e-07,0.067324,0.0
103,Zangaglia 2009,0.0,STN_BL,32.0,Improved,0.099419,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,NaN,...,-0.080103,-5.237877e-07,-0.040052,0.0,0.001943,0.001965,0.134648,9.750566e-07,0.067324,0.0


In [31]:
# Perform an outer join on dmg_df and data_df using the 'path' column
merged_df = data_df.merge(dmg_df, on='path', how='outer')
merged_df

,study,Memory_Specific,ses,N,Memory_Outcome,Pre_Post_Memory_Effect_Size__Cohen_s_D_,Location,List_of_Coordinates,Side,Frequency__Hz_,...,agreement_pcc_inverse_r_map_sum,agreement_pcc_inverse_r_map_average_subject_in_target,agreement_pcc_inverse_r_map_average_target_in_subject,agreement_pcc_inverse_r_map_num_in_roi,agreement_Precom_T_spatial_corr,agreement_Precom_T_cosine,agreement_Precom_T_sum,agreement_Precom_T_average_subject_in_target,agreement_Precom_T_average_target_in_subject,agreement_Precom_T_num_in_roi
0,Acera 2019,0.0,STN_BL,50.0,Impaired,-0.180960,STN,"[14.2, -15.1, -6.1], [-13.5, -14.2, -6.1]",BL,NaN,...,-0.039039,-2.826996e-07,-0.019519,0.0,-0.000435,-0.000424,-7.421073,-0.000053,-3.710537,0.0
1,Asahi 2014,0.0,STN_BL,11.0,Improved,0.234145,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,130.0,...,0.134648,9.750566e-07,0.067324,0.0,0.001466,0.001477,25.829446,0.000184,12.914723,0.0
2,Barcia 2022,1.0,Fornix_BL,1.0,Mixed,NaN,Fornix,"[-2, 3.5, -1], [2, 3.5, -1]",BL,NaN,...,0.326355,2.363317e-06,0.163178,0.0,0.004245,0.004256,74.431358,0.000530,37.215679,0.0
3,Bergfeld 2017,0.0,vALIC_BL,20.0,Impaired,-0.688208,vALIC,"[-11.6, 7.2, -3.2], [12.7, 8.7, -2.4]",BL,140.0,...,-0.037524,-2.717330e-07,-0.018762,0.0,-0.000229,-0.000218,-3.807755,-0.000027,-1.903878,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.134648,9.750566e-07,0.067324,0.0,0.001466,0.001477,25.829446,0.000184,12.914723,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.039039,-2.826996e-07,-0.019519,0.0,-0.000435,-0.000424,-7.421073,-0.000053,-3.710537,0.0
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.134648,9.750566e-07,0.067324,0.0,0.001466,0.001477,25.829446,0.000184,12.914723,0.0
102,You 2020,0.0,STN_BL,NaN,NaN,NaN,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,NaN,...,0.134648,9.750566e-07,0.067324,0.0,0.001466,0.001477,25.829446,0.000184,12.914723,0.0
103,Zangaglia 2009,0.0,STN_BL,32.0,Improved,0.099419,STN,"[-16.7, -6.0, -1.7], [18.6, -5.3, -1.7]",BL,NaN,...,0.134648,9.750566e-07,0.067324,0.0,0.001466,0.001477,25.829446,0.000184,12.914723,0.0


In [32]:
merged_df.to_csv(master_path, index=False)

All done. Enjoy your analyses. 

--Calvin 